In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import selenium
print(selenium.__version__)

4.8.2


# 4Chan

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
# browser.set_window_position(1920,0)
browser.get("https://boards.4channel.org/g/thread/92468569#p92471725")
# href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
# href = href.split("&url=")[1]
# browser.get(href)
# browser.close()
# del browser
# print(f"href: {href}")

In [17]:
result = browser.execute_script(r"return {a:4, b:6, myName:'is daddy'};")
print(result) # holy shit it converts the objects

{'a': 4, 'b': 6, 'myName': 'is daddy'}


In [ ]:
import requests
url = "http://127.0.0.1:5000/"
#     prompt = f"""The following is a conversation between a human and an AI.
# Human: {human_input}
# AI:"""
prompt=f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{human_input}

### Context:
{human_input}

### Response:"""
data = {"prompts": [prompt]}
response = requests.post(url, json=data).json()[0]
response = response.replace(prompt, "")
if "Human:" in response:
    response = response.split("Human:")[0]
response = response.strip()
return response
# await get_response("Hi there!")

# Picker

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Edge()
driver.get("https://twitter.com")

In [80]:
async def getSelector(thing: str):
    print("Prompting user to click")
    result = driver.execute_async_script(open("picker.js","r").read())
    print(f"Found element with selector {result}")

    import requests
    url = "http://127.0.0.1:5000/"
    PROMPT_DICT = {
        "prompt_input": (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
        ),
        "prompt_no_input": (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:"
        ),
    }
    def fmt(prompt: str, input: str = "") -> str:
        if input.strip() == "":
            return PROMPT_DICT['prompt_no_input'].format_map({'instruction': prompt})
        else:
            return PROMPT_DICT['prompt_input'].format_map({'instruction': prompt, 'input': input})
        
    prompts=[
    # fmt(
    #     """You are a calibration engine for a web interface.
    # The user has just indicated the log in button on a webpage.
    # Clean up the CSS selector for the log in button.
    # Remove any unnecessary parts, and make sure it is as short as possible.
    # You do not need to start at the root element.
    # Try and make it as short as possible.

    # ### Example Input:
    # body > div#react-root > div > div:n-th-of-type(2) > div > a[href="/home"] > div

    # ### Example Output:
    # a[href="/home"]
    # """,

    #     result),
        fmt(
        """You are a calibration engine for a web interface.
The user has just indicated the {thing} on a webpage.
Clean up the XPath selector for the {thing}.
Remove any unnecessary parts, and make sure it is as short as possible.
You do not need to start at the root element.
Try and make it as short as possible.

### Example Input:
/html/body/div[@id="react-root"]/div/div[2]/div/a[@href="/home"]/div

### Example Output:
//a[@href="/home"]

### Example Input:
/./body/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./span[contains(., 'Log out')]/./span[contains(., 'Log out')]

### Example Output:
//span[text()[contains(., 'Log out')]]

### Example Input:
/./div/./input[@autocomplete='username']/./input[@autocomplete='username']

### Example Output:
//input[@autocomplete='username']
""",

        result),
    ]
    data = {"prompts": prompts}
    print("Refining selector with remote model")
    response = requests.post(url, json=data).json()
    # print(response)
    selector = response[0].split("### Response:")[1].split("###")[0].strip()
    print(f"Refined selector: {selector}")
    assert driver.find_element(By.XPATH, selector) is not None
    return selector


In [81]:
from time import sleep

In [77]:
def click(xpath):
    driver.execute_script(f"""
        document.evaluate(
            `{xpath}`, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null
        ).singleNodeValue.click();
    """)

In [112]:
def focus(xpath):
    """
    1. Insert a panel into the page to absorb a mouse click
    2. Click the browser content area to allow for programmatic focusing of elements (browser security thing)
    3. The panel removes itself
    4. The element is focused
    """
    import pyautogui
    driver.execute_script(r"""
const panel = document.createElement("div");
panel.style.position = "fixed";
panel.style.top = "0";
panel.style.left = "0";
panel.style.width = "100%";
panel.style.height = "100%";
panel.style.zIndex = "9999";
panel.style.background = "rgba(0,0.5,0,0.5)";
panel.innerText = "CLICK FOR FOCUS";
panel.style.color = "white";
panel.style.fontSize = "50px";
panel.style.fontFamily = "Impact, Charcoal, sans-serif";

panel.style.display = "flex";
panel.style.justifyContent = "center";
panel.style.alignItems = "center";
panel.style.cursor = "pointer";
document.body.appendChild(panel);
panel.addEventListener("click", () => {
    document.body.removeChild(panel);
});
""")
    # click on the browser window
    restore = pyautogui.position()
    pyautogui.click(driver.get_window_position()['x'] + driver.get_window_size()["width"]//4, driver.get_window_position()['y'] + driver.get_window_size()["height"]//2)
    pyautogui.moveTo(restore)

    driver.execute_script(f"""
        document.evaluate(
            `{xpath}`, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null
        ).singleNodeValue.focus();
    """)


In [68]:
logInSelector = await getSelector("log in button")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./div/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]/./span[contains(., 'Log in')]
Refining selector with remote model
Refined selector: //*[text()[contains(., 'Log in')]]


In [69]:
click(logInSelector)

In [82]:
usernameSelector = await getSelector("username input")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div[@id='layers']/./div/./div/./div/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./div/./div/./div/./input[@autocomplete='username']/./input[@autocomplete='username']/./input[@autocomplete='username']
Refining selector with remote model
Refined selector: //input[@autocomplete='username']


In [113]:
focus(usernameSelector)

In [130]:
def didOnePassOpen():
    import mss
    import pyautogui
    from IPython.display import display
    from PIL import Image
    def getMouseMonitorScreenshot():
        # Get the current mouse position
        mouse_x, mouse_y = pyautogui.position()

        # Determine the monitor the mouse is on
        with mss.mss() as sct:
            target_monitor = None
            for monitor in sct.monitors[1:]:
                if monitor["left"] <= mouse_x < monitor["left"] + monitor["width"] and monitor["top"] <= mouse_y < monitor["top"] + monitor["height"]:
                    target_monitor = monitor
                    break

            if target_monitor:
                image = sct.grab(target_monitor)
                return Image.frombytes("RGB", image.size, image.bgra, "raw", "BGRX")
            else:
                return None
    import pytesseract
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    def recognize_text(image):
        return pytesseract.image_to_string(image, lang="eng")
    
    
    text = recognize_text(getMouseMonitorScreenshot())
    # print("OCR", text)
    if "Search 1Password" in text or "Q earch 1Password" in text:
        return True
    elif "Copy Username" in text and "Copy Password" in text:
        return True
    else:
        return False


In [116]:
import pyautogui
pyautogui.hotkey("ctrl","shift","space")
sleep(0.5)
if didOnePassOpen():
    pyautogui.typewrite("twitter")
    sleep(0.5)
    pyautogui.hotkey("ctrl","c")
    focus(usernameSelector)
    sleep(0.5)
    pyautogui.hotkey("ctrl","a")
    pyautogui.hotkey("ctrl","v")
    

In [87]:
loginNextSelector = await getSelector("next button")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div[@id='layers']/./div/./div/./div/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./span[contains(., 'Next')]/./span[contains(., 'Next')]
Refining selector with remote model
Refined selector: //span[text()[contains(., 'Next')]]


In [117]:
click(loginNextSelector)

In [89]:
passwordSelector = await getSelector("password input")

Prompting user to click
Found element with selector /./body/./div/./div/./div/./div[@id='layers']/./div/./div/./div/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div[@role='group']/./div/./div/./div/./div/./div/./div/./div/./div/./div/./input[@autocomplete='current-password']
Refining selector with remote model
Refined selector: //input[@autocomplete='current-password']


In [119]:
focus(passwordSelector)

In [132]:
pyautogui.hotkey("ctrl","shift","space")
sleep(0.5)
if didOnePassOpen():
    pyautogui.typewrite("twitter")
    sleep(0.5)
    pyautogui.hotkey("ctrl","shift","c")
    focus(passwordSelector)
    sleep(0.5)
    pyautogui.hotkey("ctrl","a")
    pyautogui.hotkey("ctrl","v")
else:
    raise Exception("1Password not open")

In [133]:
loginSelector = await getSelector("dialog login button")

Prompting user to click


In [94]:
click(loginSelector)

# Tweet gatherer

In [ ]:
program = """
Go to twitter.com.
Check if the user is logged in. If there is """

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Edge()
driver.get("https://twitter.com")


# Forge MDK

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options as EdgeOptions
from pathlib import Path

downloads_dir = Path("downloads")
downloads_dir.mkdir(exist_ok=True)
# from msedge.selenium_tools import EdgeOptions
options = EdgeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": str(downloads_dir),
    "download.directory_upgrade": True,
})
# options.add_argument("--headless")
# options.add_argument("--disable-gpu") 
browser = webdriver.Edge(options=options)
browser.set_window_position(1920,0)
browser.get("https://files.minecraftforge.net/")
href = browser.find_element(By.CSS_SELECTOR, "a[title='Mdk']").get_attribute("href")
href = href.split("&url=")[1]
browser.get(href)
# browser.close()
# del browser
print(f"href: {href}")

href: https://maven.minecraftforge.net/net/minecraftforge/forge/1.19.3-44.1.23/forge-1.19.3-44.1.23-mdk.zip


In [ ]:
%cd ../../src
from tools.forge_mdk import ForgeMDKTool
from tools.helpers import handle_tool_invocation
# tool = ForgeMDKTool()
# tool.invoke("latest")
await handle_tool_invocation("[EDGE(open https://files.minecraftforge.net/) ->")

d:\Repos\Other\toolformer\backend\src
hehe


Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packages\selenium\webdriver\common\service.py", line 72, in start
    self.process = subprocess.Popen(cmd, env=self.env,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\TeamD\.conda\envs\3.10\lib\subprocess.py", line 1440, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
FileNotFoundError: [WinError 2] The system cannot find the file specified

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Repos\Other\toolformer\backend\src\tools\helpers.py", line 35, in handle_tool_invocation
    result = tool.method(params)
  File "d:\Repos\Other\toolformer\backend\src\tools\edge.py", line 31, in invoke
    driver = Edge(options=options)
  File "d:\Repos\Other\toolformer\backend\venv\lib\site-packa

(True,
 "ERROR: Message: 'msedgedriver' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687\n")